In [12]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
billboard_csv = '../Resources/billboard_lyrics_1964-2015.csv'
song_list_csv = '../Resources/song_info.csv'

In [3]:
# set dataframes from csv files
billboard_df = pd.read_csv(billboard_csv, encoding = 'ISO-8859-1')
billboard_df.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [5]:
song_df = pd.read_csv(song_list_csv)
song_df.head()

,song_name,artist_name,album_names,playlist
0,Boulevard of Broken Dreams,Green Day,Greatest Hits: God's Favorite Band,00s Rock Anthems
1,In The End,Linkin Park,Hybrid Theory,00s Rock Anthems
2,Seven Nation Army,The White Stripes,Elephant,00s Rock Anthems
3,By The Way,Red Hot Chili Peppers,By The Way (Deluxe Version),00s Rock Anthems
4,How You Remind Me,Nickelback,Silver Side Up,00s Rock Anthems


In [6]:
# add columns to both dfs with lowercase song names
billboard_df['fuzzy_name'] = billboard_df['Song'].apply(lambda x: x.lower())
song_df['fuzzy_name'] = song_df['song_name'].apply(lambda x: x.lower())

In [17]:
# remove punctuation from lowercase names
trans_table = str.maketrans('','',string.punctuation)
billboard_df['fuzzy_name'] = billboard_df['fuzzy_name'].apply(lambda x: x.translate(trans_table))
song_df['fuzzy_name'] = song_df['fuzzy_name'].apply(lambda x: x.translate(trans_table))

In [20]:
# used song_df.tail as verification to see song # 18832 which has parentheses removed
song_df.tail(10)

,song_name,artist_name,album_names,playlist,fuzzy_name
18825,Something Familiar,Josienne Clarke and Ben Walker,Overnight,Your Favorite Coffeehouse,something familiar
18826,Call It Dreaming,Iron & Wine,Beast Epic,Your Favorite Coffeehouse,call it dreaming
18827,Stay Awake,Joseph,Stay Awake,Your Favorite Coffeehouse,stay awake
18828,Build Me Up From Bones,Sarah Jarosz,Build Me Up From Bones,Your Favorite Coffeehouse,build me up from bones
18829,I Know,Sammy Brue,I Am Nice,Your Favorite Coffeehouse,i know
18830,Let It Breathe,Water Liars,Water Liars,Your Favorite Coffeehouse,let it breathe
18831,Answers,Brooke Annibale,The Simple Fear,Your Favorite Coffeehouse,answers
18832,Sudden Love (Acoustic),The Woodlands,"Parallels, Vol. II",Your Favorite Coffeehouse,sudden love acoustic
18833,Gentle on My Mind,Robert Ellis,Dear John,Your Favorite Coffeehouse,gentle on my mind
18834,Up to Me,Sera Cahoone,Up to Me,Your Favorite Coffeehouse,up to me


In [61]:
# fuzzy testing
query = billboard_df['fuzzy_name'][2]
choices = song_df['fuzzy_name']
# yields only top result
result = process.extractOne(query,choices)
result

('i cant get no satisfaction', 100, 251)

In [62]:
# top 5 results
results = process.extract(query,choices)
results

[('i cant get no satisfaction', 100, 251),
 ('i can', 90, 3751),
 ('satisfaction', 90, 5709),
 ('i can', 90, 9356),
 ('cant stop', 86, 13)]

In [63]:
# double check information
song_df.loc[251]

song_name           (I Can't Get No) Satisfaction
artist_name                    The Rolling Stones
album_names    Hot Rocks (1964-1971) [Remastered]
playlist                         60s Rock Anthems
fuzzy_name             i cant get no satisfaction
Name: 251, dtype: object

In [71]:
# will reference only 2010-2015 data
result[1]


100

In [72]:
# loop through billboard df
query_name = []
matched_name = []
ratio_value = []
song_index = []
fuzzy_list = []
choices = song_df['fuzzy_name']
for i in range(len(billboard_df)):
    query = billboard_df['fuzzy_name'][i]
    query_name.append(billboard_df['fuzzy_name'][i])
#     matched_name.append(process.extractOne(query,choices)[0])
#     ratio_value.append(process.extractOne(query,choices)[1])
#     song_index.append(process.extractOne(query,choices)[2])
    fuzzy_list.append(process.extractOne(query,choices))

In [80]:
fuzzy_list[2][0]

'i cant get no satisfaction'

In [82]:
for i in range(len(fuzzy_list)):
    matched_name.append(fuzzy_list[i][0])
    ratio_value.append(fuzzy_list[i][1])
    song_index.append(fuzzy_list[i][2])

In [88]:
fuzzy_dict = {'Song Name':query_name,
             'Matched Name':matched_name,
             'Ratio Value':ratio_value,
             'Song List Index':song_index}
fuzzy_df = pd.DataFrame(fuzzy_dict)

In [89]:
fuzzy_df

,Song Name,Matched Name,Ratio Value,Song List Index
0,wooly bully,wo,90,12363
1,i cant help myself sugar pie honey bunch,i cant help myself sugar pie honey bunch,100,3863
2,i cant get no satisfaction,i cant get no satisfaction,100,251
3,you were on my mind,youre on my mind,91,11156
4,youve lost that lovin feelin,lost,90,4415
...,...,...,...,...
5095,el perdon,el perdón,94,10269
5096,she knows,now,90,10519
5097,night changes,change,90,1815
5098,back to back,back to back,100,3684
